In [ ]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
import keras as keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras import optimizers
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.layers import GRU, LSTM, Activation, Bidirectional
from keras.layers import Dense, Embedding
from keras.layers import TimeDistributed
from keras.layers import Concatenate, Flatten 
from keras.layers import Dense, concatenate
import time
import matplotlib.pyplot as plt
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from numpy import dstack
#from attention import Attention

# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import ConvLSTM2D
from scipy.optimize import differential_evolution
from sklearn.datasets import make_blobs

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#For Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import csv

In [ ]:
tf.debugging.set_log_device_placement(True)

### Predeined Parameters and Blocks

In [ ]:
vocab_size = 1000
embed_dim =8 # Embedding size for each token
maxlen = 50
num_heads = 2  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer
trunc_type = 'post'
padding_type ='post'
oov_tok = "<OOV>"

num_epochs = 100

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        
    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
        #return x

In [ ]:
class TokenEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        #return x + positions
        return x

In [ ]:
class TokenAndPositionAndDurationEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionAndDurationEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.dur_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x, y):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        y = self.dur_emb(y)
        return x + y + positions

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): 
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_' + string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+ string])
  plt.show()

avg_len=0
with open('Window/Chatting_activity__AR.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
sentences1 = np.empty(len(data), dtype=object)

for i in range(len(data)):
    if(len(data[i])>maxlen):
        maxlen =len(data[i])
    avg_len +=len(data[i])
    sentences1[i] =' '.join(data[i])
print(len(sentences1))

with open('Window/Presentation_activity__AR.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
sentences2 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    if(len(data[i])>maxlen):
        maxlen =len(data[i])
    avg_len +=len(data[i])
    sentences2[i] =' '.join(data[i])
print(len(sentences2))


with open('Window/Discussion_activity__AR.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
sentences3 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    if(len(data[i])>maxlen):
        maxlen =len(data[i])
    avg_len +=len(data[i])
    sentences3[i] =' '.join(data[i])
print(len(sentences3))

with open('Window/GroupStudy_activity__AR.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
sentences4 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    if(len(data[i])>maxlen):
        maxlen =len(data[i])
    avg_len +=len(data[i])
    sentences4[i] =' '.join(data[i])
print(len(sentences4))

#Combine all np arrays
sentences =  np.concatenate((sentences1, sentences2))
sentences =  np.concatenate((sentences, sentences3))
sentences =  np.concatenate((sentences, sentences4))
sentences = sentences.tolist()
print(len(sentences))

# Construct Y label
labels = [] #  empty regular list
for i in range(len(sentences1)):
    labels.append(0*np.ones((1)))
for i in range(len(sentences2)):
    labels.append(1*np.ones((1)))
for i in range(len(sentences3)):
    labels.append(2*np.ones((1)))
for i in range(len(sentences4)):
    labels.append(3*np.ones((1)))
#np_array = np.array(arr)  # transformed to a numpy array
#y_df = pd.DataFrame({'': np_array[:, 0]})
#maxlen = 100
#maxlen = (int)(avg_len/len(sentences))
print(len(labels))
print(maxlen)

In [ ]:
with open('Window/Chatting_duration_AR_v2.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
duration1 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    duration1[i] =' '.join(data[i])
print(len(duration1))

with open('Window/Presentation_duration_AR_v2.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
duration2 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    duration2[i] =' '.join(data[i])
print(len(duration2))


with open('Window/Discussion_duration_AR_v2.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
duration3 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    duration3[i] =' '.join(data[i])
print(len(duration3))

with open('Window/GroupStudy_duration_AR_v2.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
duration4 = np.empty(len(data), dtype=object)
for i in range(len(data)):
    duration4[i] =' '.join(data[i])
print(len(duration4))

#Combine all np arrays
duration =  np.concatenate((duration1,duration2))
duration =  np.concatenate((duration, duration3))
duration =  np.concatenate((duration, duration4))
duration =  duration.tolist()
print(len(duration))

### Pre-defined parameters and models